In [2]:
import os
import glob
import csv
import numpy as np
import pandas as pd
from datetime import datetime
import pytz
from typing import Optional, Tuple, List

from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, confusion_matrix
from sklearn.impute import SimpleImputer


pasta = r"C:\Users\alisson.silva\D\audaces\BI\__Bi_Comercial\previsao_churn"

# Sinais
N_RECENT = 3
N_BASE   = 6
CAP_DIAS = 180
GAMMA    = 0.7
W_MOMENTO, W_TEND, W_REC = 0.50, 0.35, 0.15
ALPHA = 0.5               
INFLUENCIA_TICKET = 0.15    
W_NPS = 0.15               
NEUTRO_NPS = 0.30          


K_PRAZO_MESES = 2
TARGET_RECALL = 0.40         
THR_GRID = np.linspace(0.20, 0.60, 21) 

# Data "de hoje" determinística (timezone do usuário)
tz = pytz.timezone("Europe/Madrid")
today = pd.Timestamp(datetime.now(tz).date())

# Utils

def classificar_risco_cont(v: float) -> str:
    if v < 0.3: return "Baixo Risco"
    if v < 0.7: return "Risco Médio"
    return "Alto Risco"

def classificar_eng(v: float) -> str:
    if v < 0.33: return "Baixo Engajamento"
    if v < 0.66: return "Engajamento Médio"
    return "Alto Engajamento"

def risco_nps(n):
    if pd.isna(n): return np.nan
    n = float(n)
    if n >= 9:   return 0.0   
    if n >= 7:   return 0.20  
    return 0.60                


def peso_ticket(v):
    if v < 1000:  return 1
    if v < 2500:  return 2
    if v < 4000:  return 3
    return 4

def fbeta_score_from_pr(precision: float, recall: float, beta: float = 2.0) -> float:
    """Calcula F-beta a partir de P e R."""
    beta2 = beta**2
    denom = (beta2 * precision + recall)
    if denom == 0: return 0.0
    return (1 + beta2) * (precision * recall) / (denom + 1e-12)

def pick_threshold(probas: np.ndarray, y_true: np.ndarray,
                   thr_grid: np.ndarray = THR_GRID,
                   target_recall: float = TARGET_RECALL,
                   beta: float = 2.0) -> Tuple[float, pd.DataFrame]:
    """Escolhe threshold priorizando recall (via F-beta alto e meta de recall)."""
    rows = []
    for thr in thr_grid:
        pred = (probas >= thr).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        fbeta     = fbeta_score_from_pr(precision, recall, beta=beta)
        acc       = (tp + tn) / (tp + tn + fp + fn)
        rows.append({"thr":thr, "precision":precision, "recall":recall, "fbeta":fbeta, "accuracy":acc,
                     "tp":tp, "fp":fp, "fn":fn, "tn":tn})
    df_res = pd.DataFrame(rows).sort_values(["recall","fbeta","precision"], ascending=[False,False,False])

    # 1) tenta cumprir a meta de recall com maior fbeta
    cand = df_res[df_res["recall"] >= target_recall]
    if not cand.empty:
        best = cand.sort_values(["fbeta","precision"], ascending=[False,False]).iloc[0]
        return float(best["thr"]), df_res

    # 2) se não cumprir meta, pega o maior fbeta geral
    best = df_res.sort_values(["fbeta","recall"], ascending=[False,False]).iloc[0]
    return float(best["thr"]), df_res

# 1) Ler base e normalizar

def ler_base_unificada(pasta: str) -> pd.DataFrame:
    arquivos = glob.glob(os.path.join(pasta, "*.xlsx"))
    if not arquivos:
        raise FileNotFoundError("Nenhum .xlsx encontrado na pasta base.")

    df = pd.concat([pd.read_excel(a) for a in arquivos], ignore_index=True)

    # Tipos/limpeza
    for c in ['Software','Package','Country','CardCode','CardName','State','email']:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()

    df['created']   = pd.to_datetime(df.get('created'), errors='coerce')
    df['Start']     = pd.to_datetime(df.get('Start'),   errors='coerce')
    df['Contagem']  = pd.to_numeric(df.get('Contagem'), errors='coerce').fillna(0).astype(int)

    df['ano'] = pd.to_numeric(df.get('ano'), errors='coerce').astype('Int64')
    df['mes'] = pd.to_numeric(df.get('mes'), errors='coerce').astype('Int64')

    return df


# 2) Painel mensal completo (corrigido e robusto)

def painel_mensal(df: pd.DataFrame) -> pd.DataFrame:
    base = (
        df.dropna(subset=['ano','mes'])
          .assign(
              ano=lambda x: x['ano'].astype(int),
              mes=lambda x: x['mes'].astype(int),
              Contagem=lambda x: pd.to_numeric(x['Contagem'], errors='coerce').fillna(0)
          )
          .loc[:, ['CardCode','ano','mes','Contagem']]
    )

    if base.empty:
        raise ValueError("Base mensal está vazia após limpeza de ano/mes/Contagem.")

    base['period'] = pd.PeriodIndex(year=base['ano'], month=base['mes'], freq='M')

    # Agrega por CardCode x period
    agg = (
        base.groupby(['CardCode','period'], as_index=False)['Contagem']
            .sum()
            .sort_values(['CardCode','period'])
    )


    out_rows = []
    for card, g in agg.groupby('CardCode'):
        pmin, pmax = g['period'].min(), g['period'].max()
        full_p = pd.period_range(pmin, pmax, freq='M')
        gi = (
            g.set_index('period')[['Contagem']]
             .reindex(full_p)
             .fillna({'Contagem': 0})
             .rename_axis('period')
             .reset_index()
        )
        gi['CardCode'] = card
        out_rows.append(gi)

    full = pd.concat(out_rows, ignore_index=True)

  
    ts = full['period'].dt.to_timestamp(how='start')
    full['ano'] = ts.dt.year.astype(int)
    full['mes'] = ts.dt.month.astype(int)
    full['dt']  = pd.to_datetime(full['ano'].astype(str) + '-' + full['mes'].astype(str) + '-01')

    full = full[['CardCode','ano','mes','Contagem','dt']].sort_values(['CardCode','dt']).reset_index(drop=True)

    if full.empty:
        raise ValueError("painel_mensal gerou vazio — verifique colunas CardCode/ano/mes/Contagem.")
    return full

# 3) Estatísticas por cliente

def resumo_clientes(df: pd.DataFrame) -> pd.DataFrame:
    dfc = df.groupby('CardCode').agg({
        'Contagem': 'sum',
        'created':  ['min','max','nunique']
    }).reset_index()
    dfc.columns = ['CardCode','total_acessos','primeiro_acesso','ultimo_acesso','dias_com_acesso']
    dfc['tempo_ativo'] = (dfc['ultimo_acesso'] - dfc['primeiro_acesso']).dt.days.clip(lower=0)

    # Recência (dias desde último acesso)
    dias = (today - dfc['ultimo_acesso']).dt.days
    if dias.notna().any():
        max_fallback = dias.dropna().max()
    else:
        max_fallback = CAP_DIAS
    dfc['dias_desde_ultimo'] = dias.fillna(max_fallback).clip(lower=0)

 
    for col in ['total_acessos','dias_com_acesso','tempo_ativo']:
        q95 = dfc[col].quantile(0.95)
        dfc[col] = dfc[col].clip(upper=q95)

    sc_eng = MinMaxScaler()
    dfc[['acessos_norm','dias_norm','tempo_norm']] = sc_eng.fit_transform(
        dfc[['total_acessos','dias_com_acesso','tempo_ativo']].astype(float)
    )
    dfc['engajamento_score'] = (
        0.5*dfc['acessos_norm'] + 0.3*dfc['dias_norm'] + 0.2*dfc['tempo_norm']
    ).clip(0,1)

    return dfc

# 4) Tendência e queda recentes

def slope_stats(g: pd.DataFrame):
    g = g.sort_values(['ano','mes'])
    x = (g['ano']*12 + g['mes']).astype(float).to_numpy()
    y = g['Contagem'].astype(float).to_numpy()

    mean_y = np.mean(y)
    std_y  = np.std(y, ddof=1) if len(y) > 1 else 0.0

    if len(x) < 2 or np.all(y == y[0]):
        slope_abs = 0.0
    else:
        slope_abs = float(linregress(x, y).slope)

    eps = 1e-6
    slope_per_mean = slope_abs / (mean_y + eps)
    slope_per_std  = slope_abs / (std_y  + eps)

    return pd.Series({
        'media_cliente': mean_y,
        'std_cliente': std_y,
        'slope_abs': slope_abs,
        'slope_per_mean': slope_per_mean,
        'slope_per_std': slope_per_std
    })

def recent_vs_baseline(g: pd.DataFrame):
    g = g.sort_values(['ano','mes'])
    y = g['Contagem'].astype(float).to_numpy()
    if len(y) < (N_RECENT + N_BASE):
        return pd.Series({'recent_mean': np.nan, 'baseline_mean': np.nan, 'queda_perc': np.nan})
    recent = float(np.mean(y[-N_RECENT:]))
    base   = float(np.mean(y[-(N_RECENT+N_BASE):-N_RECENT]))
    eps = 1e-6
    queda = (base - recent) / (base + eps)
    return pd.Series({'recent_mean': recent, 'baseline_mean': base, 'queda_perc': queda})


# 5) Ticket (opcional) e NPS (opcional)

def ler_ticket(pasta: str) -> pd.DataFrame:
    caminho = os.path.join(pasta, "tkt_medio", "ticket_medio.csv")
    if not os.path.exists(caminho):
        return pd.DataFrame(columns=['CardCode','TicketMedio','peso_ticket','peso_norm'])
    try:
        try:
            df = pd.read_csv(caminho)
        except Exception:
            df = pd.read_csv(caminho, sep=';', decimal=',', quoting=csv.QUOTE_MINIMAL)
        df.columns = [c.strip() for c in df.columns]
        df['CardCode'] = df['CardCode'].astype(str).str.strip()
        df['TicketMedio'] = pd.to_numeric(df['TicketMedio'], errors='coerce').fillna(0)
        df['peso_ticket'] = df['TicketMedio'].apply(peso_ticket)
        df['peso_norm'] = ((df['peso_ticket'] - 1) / 3).fillna(0.0)
        return df[['CardCode','TicketMedio','peso_ticket','peso_norm']]
    except Exception:
        return pd.DataFrame(columns=['CardCode','TicketMedio','peso_ticket','peso_norm'])

def ler_nps(pasta: str) -> pd.DataFrame:
    caminho = os.path.join(pasta, "NPS.xlsx")
    if not os.path.exists(caminho):
        return pd.DataFrame(columns=['CardCode','nota_nps','risco_nps'])
    try:
        df = pd.read_excel(caminho)
        df.columns = [c.strip() for c in df.columns]
        df = df.rename(columns={'NPS':'nota_nps'})
        df['CardCode'] = df['CardCode'].astype(str).str.strip()  # <<< corrigido
        df['nota_nps'] = pd.to_numeric(df['nota_nps'], errors='coerce')
        df['risco_nps'] = df['nota_nps'].apply(risco_nps)
        return df[['CardCode','nota_nps','risco_nps']]
    except Exception:
        return pd.DataFrame(columns=['CardCode','nota_nps','risco_nps'])

# 6) Score por regra (online)

def score_regra(df_panel: pd.DataFrame, dfc: pd.DataFrame, df_ticket: pd.DataFrame, df_nps: pd.DataFrame) -> pd.DataFrame:
    if df_panel is None or df_panel.empty:
        raise ValueError("score_regra recebeu df_panel vazio.")

    # Tendência
    trend = df_panel.groupby('CardCode', as_index=False).apply(slope_stats)
    rb    = df_panel.groupby('CardCode', as_index=False).apply(recent_vs_baseline)

    base = (dfc.merge(trend, on='CardCode', how='left')
              .merge(rb, on='CardCode', how='left'))

   
    base['queda_perc'] = base['queda_perc'].fillna(0).clip(-1, 1)
    base['slope_per_mean'] = base['slope_per_mean'].fillna(0)

    base['momento_queda'] = base['queda_perc'].clip(0, 1)
    base['tend_rel_inv']  = (-base['slope_per_mean']).clip(lower=0)

  
    base['recencia_raw'] = base['dias_desde_ultimo'].astype(float).clip(0, CAP_DIAS)
    base['recencia_n']   = (base['recencia_raw'] / CAP_DIAS).clip(0, 1)

    
    _sc = MinMaxScaler()
    base[['momento_queda_n','tend_rel_inv_n']] = _sc.fit_transform(
        base[['momento_queda','tend_rel_inv']].to_numpy().astype(float)
    )

    
    base['momento_queda_n_b'] = np.power(base['momento_queda_n'], GAMMA)
    base['tend_rel_inv_n_b']  = np.power(base['tend_rel_inv_n'],  GAMMA)
    base['recencia_n_b']      = np.power(base['recencia_n'],      GAMMA)

   
    base['risco_raw'] = (
        W_MOMENTO*base['momento_queda_n_b'] +
        W_TEND   *base['tend_rel_inv_n_b']  +
        W_REC    *base['recencia_n_b']
    ).clip(0,1)

    
    rank_pct = base['risco_raw'].rank(pct=True)
    rank_pct_boost = np.power(rank_pct, 0.85)
    base['risco_blend'] = (ALPHA*base['risco_raw'] + (1-ALPHA)*rank_pct_boost).clip(0,1)

    if not df_ticket.empty:
        base = base.merge(df_ticket[['CardCode','peso_norm']], on='CardCode', how='left')
        base['peso_norm'] = base['peso_norm'].fillna(0.0)
    else:
        base['peso_norm'] = 0.0

    base['risco_churn_novo'] = (base['risco_blend'] + INFLUENCIA_TICKET*base['peso_norm']).clip(0,1)

    
    base['nivel_engajamento'] = base['engajamento_score'].apply(classificar_eng)

    
    if not df_nps.empty:
        base = base.merge(df_nps[['CardCode','nota_nps','risco_nps']], on='CardCode', how='left')
    else:
        base['nota_nps'] = np.nan
        base['risco_nps'] = np.nan

    
    base[['media_cliente','std_cliente','momento_queda','tend_rel_inv','recencia_n']] = \
        base[['media_cliente','std_cliente','momento_queda','tend_rel_inv','recencia_n']].fillna(0.0)

    base['coef_var'] = (base['std_cliente'] / (base['media_cliente'] + 1e-6)).fillna(0.0)
    base['tempo_ativo_meses'] = (base.get('tempo_ativo', pd.Series(0, index=base.index)).astype(float).fillna(0.0) / 30.0)
    base['queda_intensa'] = (base['momento_queda'] * (base['baseline_mean'].replace(0, np.nan).fillna(0) > 0).astype(float)).fillna(0.0)

    base['nps_risco_fill'] = base['risco_nps'].fillna(NEUTRO_NPS)
    base['engaj_score']    = base.get('engajamento_score', pd.Series(0.0, index=base.index)).fillna(0.0)

    
    base['risco_churn_final_regra'] = (
        (1 - W_NPS)*base['risco_churn_novo'] + W_NPS*base['nps_risco_fill']
    ).clip(0,1)

    base['nivel_risco_regra'] = base['risco_churn_final_regra'].apply(classificar_risco_cont)
    base['categoria_churn_regra'] = base['nivel_risco_regra'] + " / " + base['nivel_engajamento']

    return base


# 7) Backtest + ML (melhorado)

def tentar_treinar_modelo(df_panel: pd.DataFrame, df_regra: pd.DataFrame, pasta: str) -> Optional[Pipeline]:
    """Treina um modelo supervisionado com features enriquecidas e tuning de limiar.
       Retorna pipeline com atributos:
         - thr_opt_ : threshold ótimo (prioriza recall/F2)
         - feature_names_ : lista de features
         - df_thr_cv_ : tabela com desempenho por threshold no holdout/val
    """
    # Ler cancelamentos
    arq_cancel = os.path.join(pasta, "cancelados.xlsx")
    if not os.path.exists(arq_cancel):
        return None

    try:
        df_cancel = pd.read_excel(arq_cancel)
    except Exception:
        return None

    df_cancel.columns = [c.strip() for c in df_cancel.columns]
    if 'CardCode' not in df_cancel.columns:
        return None

    df_cancel['CardCode'] = df_cancel['CardCode'].astype(str).str.strip()
    df_cancel['dt_cancelamento'] = pd.to_datetime(df_cancel.get('dt_cancelamento'), dayfirst=True, errors='coerce')
    df_cancel['dt_aquisicao']    = pd.to_datetime(df_cancel.get('dt_aquisicao'),    dayfirst=True, errors='coerce')

    df_cancel_min = (
        df_cancel.sort_values('dt_cancelamento')
                 .groupby('CardCode', as_index=False)
                 .agg(dt_cancelamento=('dt_cancelamento','min'),
                      dt_aquisicao=('dt_aquisicao','min'))
    )

   
    map_dt_cancel = dict(zip(df_cancel_min['CardCode'], df_cancel_min['dt_cancelamento']))
    map_dt_aq     = dict(zip(df_cancel_min['CardCode'], df_cancel_min['dt_aquisicao']))

    
    map_peso_norm  = dict(zip(df_regra['CardCode'], df_regra.get('peso_norm', pd.Series(0.0, index=df_regra.index)).fillna(0.0)))
    map_nps_risco  = dict(zip(df_regra['CardCode'], df_regra.get('nps_risco_fill', pd.Series(NEUTRO_NPS, index=df_regra.index)).fillna(NEUTRO_NPS)))
    map_engaj      = dict(zip(df_regra['CardCode'], df_regra.get('engaj_score', pd.Series(0.0, index=df_regra.index)).fillna(0.0)))

    
    rows = []
    first_dt_map = df_panel.groupby('CardCode')['dt'].min().to_dict()

    for card, g in df_panel.groupby('CardCode'):
        g = g.sort_values('dt').reset_index(drop=True)
        y = g['Contagem'].astype(float).to_numpy()

        for i in range(len(g)):
            if i+1 < (N_BASE + N_RECENT):
                continue

            dt_t = g.loc[i, 'dt']

            dt_aq = map_dt_aq.get(card, None)
            if pd.notna(dt_aq) and dt_t < (dt_aq.replace(day=1)):
                continue

            xhist = g.iloc[:i+1]
            yhist = y[:i+1]

           
            xx = (xhist['dt'].dt.year*12 + xhist['dt'].dt.month).astype(float).to_numpy()
            mean_y = float(np.mean(yhist))
            std_y  = float(np.std(yhist, ddof=1)) if len(yhist) > 1 else 0.0
            if len(xx) < 2 or np.all(yhist == yhist[0]):
                slope_abs = 0.0
            else:
                slope_abs = float(linregress(xx, yhist).slope)
            eps = 1e-6
            slope_per_mean = slope_abs / (mean_y + eps)

           
            if len(yhist) < (N_RECENT + N_BASE):
                recent = basev = np.nan
                queda = np.nan
            else:
                recent = float(np.mean(yhist[-N_RECENT:]))
                basev  = float(np.mean(yhist[-(N_RECENT+N_BASE):-N_RECENT]))
                queda  = (basev - recent) / (basev + eps)
            queda = 0 if np.isnan(queda) else float(np.clip(queda, -1, 1))

            momento_queda = float(np.clip(queda, 0, 1))
            tend_rel_inv  = float(max(-slope_per_mean, 0))

            
            idx_pos = np.where(np.array(yhist) > 0)[0]
            if len(idx_pos) == 0:
                meses_sem_uso = np.inf
            else:
                meses_sem_uso = max((len(yhist) - 1) - idx_pos[-1], 0)
            dias_sem_uso  = 30 * (meses_sem_uso if np.isfinite(meses_sem_uso) else CAP_DIAS)
            recencia_raw  = float(np.clip(dias_sem_uso, 0, CAP_DIAS))
            recencia_n    = float(np.clip(recencia_raw / CAP_DIAS, 0, 1))

            
            coef_var = (std_y / (mean_y + 1e-6)) if mean_y > 0 else 0.0
            tempo_ativo_meses = max((dt_t - first_dt_map.get(card, dt_t)).days / 30.0, 0.0)
            queda_intensa = momento_queda * (1.0 if basev > 0 else 0.0)

            rows.append({
                'CardCode': card, 'dt': dt_t,
                'momento_queda': momento_queda, 'tend_rel_inv': tend_rel_inv, 'recencia_n': recencia_n,
                'coef_var': coef_var, 'tempo_ativo_meses': tempo_ativo_meses, 'queda_intensa': queda_intensa,
                'peso_norm': map_peso_norm.get(card, 0.0),
                'nps_risco_fill': map_nps_risco.get(card, NEUTRO_NPS),
                'engaj_score': map_engaj.get(card, 0.0)
            })

    if not rows:
        return None

    df_pred = pd.DataFrame(rows).sort_values(['dt','CardCode'])

    
    def norm_mes(gx):
        arr = gx[['momento_queda','tend_rel_inv']].to_numpy().astype(float)
        if arr.shape[0] > 1 and np.any(np.ptp(arr, axis=0) > 0):
            sc = MinMaxScaler()
            norm = sc.fit_transform(arr)
            gx['momento_queda_n'] = norm[:,0]
            gx['tend_rel_inv_n']  = norm[:,1]
        else:
            gx['momento_queda_n'] = 0.0
            gx['tend_rel_inv_n']  = 0.0

        gx['momento_queda_n_b'] = np.power(gx['momento_queda_n'], GAMMA)
        gx['tend_rel_inv_n_b']  = np.power(gx['tend_rel_inv_n'],  GAMMA)
        gx['recencia_n_b']      = np.power(gx['recencia_n'],      GAMMA)

        gx['risco_raw'] = (
            W_MOMENTO*gx['momento_queda_n_b'] +
            W_TEND   *gx['tend_rel_inv_n_b']  +
            W_REC    *gx['recencia_n_b']
        ).clip(0,1)

        rank_pct = gx['risco_raw'].rank(pct=True)
        rank_pct_boost = np.power(rank_pct, 0.85)
        gx['risco_blend'] = (ALPHA*gx['risco_raw'] + (1-ALPHA)*rank_pct_boost).clip(0,1)
        gx['risco_score'] = (gx['risco_blend'] + INFLUENCIA_TICKET*gx['peso_norm']).clip(0,1)
        return gx

    df_pred = df_pred.groupby('dt', group_keys=False).apply(norm_mes)

    
    def label_real(card, dt_t):
        dt_cancel = map_dt_cancel.get(card, pd.NaT)
        if pd.isna(dt_cancel):
            return 0
        dt_limite = (dt_t + pd.DateOffset(months=K_PRAZO_MESES)).replace(day=1) + pd.offsets.MonthEnd(0)
        return int((dt_cancel > dt_t) and (dt_cancel <= dt_limite))

    df_pred['churn_label'] = [label_real(c, d) for c, d in zip(df_pred['CardCode'], df_pred['dt'])]

   
    features = [
        'momento_queda','tend_rel_inv','recencia_n','peso_norm',
        'coef_var','tempo_ativo_meses','queda_intensa','nps_risco_fill','engaj_score'
    ]
    df_train = df_pred.dropna(subset=features).copy()
    if 'churn_label' not in df_train.columns or df_train['churn_label'].nunique() < 2:
        return None

    
    ult_dt = df_train['dt'].max()
    cut_dt = (ult_dt - pd.DateOffset(months=3)).normalize()

    X_tr = df_train.loc[df_train['dt'] <= cut_dt, features].astype(float).to_numpy()
    y_tr = df_train.loc[df_train['dt'] <= cut_dt, 'churn_label'].astype(int).to_numpy()
    X_te = df_train.loc[df_train['dt']  > cut_dt, features].astype(float).to_numpy()
    y_te = df_train.loc[df_train['dt']  > cut_dt, 'churn_label'].astype(int).to_numpy()

    
    if len(np.unique(y_tr)) < 2 or len(np.unique(y_te)) < 2:
        from sklearn.model_selection import train_test_split
        X_all = df_train[features].astype(float).to_numpy()
        y_all = df_train['churn_label'].astype(int).to_numpy()
        X_tr, X_te, y_tr, y_te = train_test_split(X_all, y_all, test_size=0.25, random_state=42, stratify=y_all)

    
    used_smote = False
    try:
        from imblearn.over_sampling import SMOTE
        sm = SMOTE(random_state=42, k_neighbors=5)
        X_tr_res, y_tr_res = sm.fit_resample(X_tr, y_tr)
        used_smote = True
    except Exception:
        X_tr_res, y_tr_res = X_tr, y_tr  # fallback sem SMOTE

    
    pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value=0.0)),  # <<< NOVO
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=500, solver='lbfgs'))
    ])
    pipe.fit(X_tr_res, y_tr_res)

    
    proba_te = pipe.predict_proba(X_te)[:,1]
   
    thr_opt, df_thr = pick_threshold(proba_te, y_te, thr_grid=THR_GRID, target_recall=TARGET_RECALL, beta=2.0)

    
    pipe.thr_opt_ = float(thr_opt)
    pipe.feature_names_ = features
    pipe.df_thr_cv_ = df_thr
    pipe.used_smote_ = used_smote
    return pipe

# 8) Pipeline principal

def main():
    # Leitura
    df_raw = ler_base_unificada(pasta)
    df_panel = painel_mensal(df_raw)

    if not isinstance(df_panel, pd.DataFrame) or df_panel.empty:
        raise ValueError("df_panel não foi gerado corretamente pelo painel_mensal().")

    dfc = resumo_clientes(df_raw)

    
    df_ticket = ler_ticket(pasta)
    df_nps    = ler_nps(pasta)

    
    df_regra = score_regra(df_panel, dfc, df_ticket, df_nps)

    
    pipe = tentar_treinar_modelo(df_panel, df_regra, pasta)

    
    if pipe is not None:
        feats = getattr(pipe, 'feature_names_', [
            'momento_queda','tend_rel_inv','recencia_n','peso_norm',
            'coef_var','tempo_ativo_meses','queda_intensa','nps_risco_fill','engaj_score'
        ])
        
        for f in feats:
            if f not in df_regra.columns:
                if f == 'nps_risco_fill':
                    df_regra[f] = df_regra.get('risco_nps', pd.Series(np.nan)).fillna(NEUTRO_NPS)
                elif f == 'engaj_score':
                    df_regra[f] = df_regra.get('engajamento_score', pd.Series(0.0)).fillna(0.0)
                else:
                    df_regra[f] = 0.0

        
        df_regra[feats] = (
            df_regra[feats]
                .assign(
                    nps_risco_fill=lambda x: x['nps_risco_fill'].fillna(NEUTRO_NPS) if 'nps_risco_fill' in x else NEUTRO_NPS,
                    engaj_score=lambda x: x['engaj_score'].fillna(0.0)               if 'engaj_score'    in x else 0.0
                )
                .fillna(0.0)
        )

        
        nan_counts = df_regra[feats].isna().sum().sort_values(ascending=False)
        if nan_counts.sum() > 0:
            print("\n⚠️ Existem NaNs nas features ao vivo:")
            print(nan_counts[nan_counts > 0].to_string())

        X_live = df_regra[feats].to_numpy(dtype=float)
        probas = pipe.predict_proba(X_live)[:,1].clip(0,1)
        df_regra['risco_ml'] = probas

        thr = float(getattr(pipe, 'thr_opt_', 0.5))
        df_regra['nivel_risco_ml'] = df_regra['risco_ml'].apply(
            lambda p: "Alto Risco" if p >= thr else ("Risco Médio" if p >= 0.3 else "Baixo Risco")
        )

        
        base_col = 'risco_churn_final_regra'
        df_regra['comparativo_dif'] = df_regra['risco_ml'] - df_regra[base_col]
    else:
       
        df_regra['risco_ml'] = np.nan
        df_regra['nivel_risco_ml'] = np.nan
        df_regra['comparativo_dif'] = np.nan

    
    cols_final = [
        'CardCode',
        'risco_churn_final_regra','nivel_risco_regra','categoria_churn_regra',
        'engajamento_score','peso_norm','nota_nps','risco_nps',
        'coef_var','tempo_ativo_meses','queda_intensa','nps_risco_fill','engaj_score',
        'risco_ml','nivel_risco_ml','comparativo_dif'
    ]
    cols_final = [c for c in cols_final if c in df_regra.columns]
    df_out = df_regra[cols_final].copy()

    # Salvar apenas o resultado final
    saida_csv_final = os.path.join(pasta, "clientes_risco_final.csv")
    df_out.to_csv(saida_csv_final, index=False, encoding='utf-8')
    print(f"✅ Arquivo final salvo em: {saida_csv_final}")
    print(f"📦 Linhas: {len(df_out):,} | Colunas: {len(df_out.columns)}")

   -    if pipe is not None:
        thr = float(pipe.thr_opt_)
        used_smote = getattr(pipe, "used_smote_", False)
        print("\n🔧 ML habilitado")
        print(f"• SMOTE usado? {'Sim' if used_smote else 'Não'}")
        print(f"• Threshold ótimo (prioriza recall / F2): {thr:.3f}")
        print("• Top thresholds (holdout):")
        print(pipe.df_thr_cv_.head(8).to_string(index=False))


if __name__ == "__main__":
    main()


# 9) Avaliação pós-processamento

import pandas as pd

try:
   
    df_pred = pd.read_csv(os.path.join(pasta, "clientes_risco_final.csv"))

   
    df_cancel = pd.read_excel(os.path.join(pasta, "cancelados.xlsx"))
    df_cancel['CardCode'] = df_cancel['CardCode'].astype(str).str.strip()

    
    df_pred['CardCode'] = df_pred['CardCode'].astype(str).str.strip()
    df_pred['cancelado'] = df_pred['CardCode'].isin(df_cancel['CardCode']).astype(int)

    label_col = 'nivel_risco_ml' if df_pred['nivel_risco_ml'].notna().any() else 'nivel_risco_regra'
    df_pred['prev_churn'] = df_pred[label_col].apply(lambda x: 1 if "Alto" in str(x) else 0)


    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(df_pred['cancelado'], df_pred['prev_churn'])
    tn, fp, fn, tp = cm.ravel()

    acuracia = (tp + tn) / (tp + tn + fp + fn)
    precisao = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall   = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1       = 2 * (precisao * recall) / (precisao + recall + 1e-9)

    print("\n📊 Resultados (pós-threshold e pós-pipeline):")
    print(f"Acurácia: {acuracia*100:.2f}%")
    print(f"Precisão: {precisao*100:.2f}%")
    print(f"Recall (sensibilidade): {recall*100:.2f}%")
    print(f"F1-Score: {f1*100:.2f}%")
    print("\nMatriz de Confusão:")
    print(f"Verdadeiros Negativos: {tn}")
    print(f"Falsos Positivos: {fp}")
    print(f"Falsos Negativos: {fn}")
    print(f"Verdadeiros Positivos: {tp}")

except Exception as e:
    print(f"\nℹ️ Avaliação final não executada (talvez sem cancelados.xlsx ou outra falha): {e}")


✅ Arquivo final salvo em: C:\Users\alisson.silva\D\audaces\BI\__Bi_Comercial\previsao_churn\clientes_risco_final.csv
📦 Linhas: 11,460 | Colunas: 16

🔧 ML habilitado
• SMOTE usado? Sim
• Threshold ótimo (prioriza recall / F2): 0.580
• Top thresholds (holdout):
 thr  precision   recall    fbeta  accuracy  tp   fp  fn    tn
0.26   0.037673 0.947368 0.162514  0.898143  90 2299   5 20226
0.24   0.035629 0.947368 0.154852  0.892087  90 2436   5 20089
0.22   0.033545 0.947368 0.146915  0.885146  90 2593   5 19932
0.20   0.031870 0.947368 0.140449  0.878912  90 2734   5 19791
0.28   0.040308 0.936842 0.171947  0.906057  89 2119   6 20406
0.30   0.043435 0.915789 0.182543  0.914943  87 1916   8 20609
0.32   0.046943 0.905263 0.194394  0.922414  86 1746   9 20779
0.34   0.049675 0.884211 0.202801  0.928470  84 1607  11 20918

📊 Resultados (pós-threshold e pós-pipeline):
Acurácia: 91.44%
Precisão: 60.71%
Recall (sensibilidade): 46.38%
F1-Score: 52.59%

Matriz de Confusão:
Verdadeiros Negativos: 9